In [ ]:
#@title ⛏️ BITCOIN 100% (CORREGIDO FINAL): SHA256d(80B) CNF universal + SAT phase-hints + PoW por target bits
!pip install python-sat -q

import hashlib
import re
import time
from pysat.solvers import Solver as PySolver

# ============================================================
# Utils Bitcoin
# ============================================================

def sha256d(b: bytes) -> bytes:
    return hashlib.sha256(hashlib.sha256(b).digest()).digest()

def bits_compact_to_target_int(bits32: int) -> int:
    exp = (bits32 >> 24) & 0xff
    mant = bits32 & 0x007fffff  # ignora sign bit como Bitcoin
    if mant == 0:
        return 0
    if exp <= 3:
        return mant >> (8 * (3 - exp))
    return mant << (8 * (exp - 3))

def target_int_to_hex_be(target_int: int) -> str:
    return target_int.to_bytes(32, 'big').hex()

def hash_display_hex(hash_bytes: bytes) -> str:
    return hash_bytes[::-1].hex()

def count_leading_zero_bits_display(hash_bytes: bytes) -> int:
    b = hash_bytes[::-1]
    count = 0
    for byte in b:
        if byte == 0:
            count += 8
        else:
            for i in range(7, -1, -1):
                if ((byte >> i) & 1) == 0:
                    count += 1
                else:
                    return count
            return count
    return count

# ============================================================
# CNF Builder: SHA256d(header80) UNIVERSAL (FIX PADDING)
#  - bits in words: LSB-first (bit0 = 2^0)
#  - header80 bytes reconstructed as word.to_bytes(4,'big')
# ============================================================

class CNFBuilderSHA256d80:
    def __init__(self):
        self.var_count = 0
        self.clauses = []
        self.var_map = {}

        self.K = [
            0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
            0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
            0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
            0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
            0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
            0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
            0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
            0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
        ]
        self.H_init = [
            0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
            0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
        ]

    def new_var(self, name=None):
        self.var_count += 1
        if name:
            self.var_map[name] = self.var_count
        return self.var_count

    def add_clause(self, lits):
        self.clauses.append(lits)

    def gate_xor(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, -c])
        self.add_clause([a, b, -c])
        self.add_clause([a, -b, c])
        self.add_clause([-a, b, c])
        return c

    def gate_and(self, a, b):
        c = self.new_var()
        self.add_clause([-a, -b, c])
        self.add_clause([a, -c])
        self.add_clause([b, -c])
        return c

    def gate_maj(self, a, b, c):
        out = self.new_var()
        self.add_clause([-a, -b, out])
        self.add_clause([-a, -c, out])
        self.add_clause([-b, -c, out])
        self.add_clause([a, b, -out])
        self.add_clause([a, c, -out])
        self.add_clause([b, c, -out])
        return out

    def gate_ch(self, e, f, g):
        out = self.new_var()
        self.add_clause([-e, -f, out])
        self.add_clause([-e, f, -out])
        self.add_clause([e, -g, out])
        self.add_clause([e, g, -out])
        return out

    def const_word(self, val):
        bits = []
        for i in range(32):
            bit_val = (val >> i) & 1
            v = self.new_var()
            self.add_clause([v] if bit_val else [-v])
            bits.append(v)
        return bits  # LSB-first

    def rot_right(self, w, n):
        return w[n:] + w[:n]

    def shift_right(self, w, n):
        zeros = []
        for _ in range(n):
            z = self.new_var()
            self.add_clause([-z])
            zeros.append(z)
        return w[n:] + zeros

    def xor_word(self, w1, w2):
        return [self.gate_xor(a, b) for a, b in zip(w1, w2)]

    def add_word(self, w1, w2):
        result = []
        carry = None
        for i in range(32):
            a = w1[i]
            b = w2[i]
            if carry is None:
                s = self.gate_xor(a, b)
                c_out = self.gate_and(a, b)
            else:
                tmp = self.gate_xor(a, b)
                s = self.gate_xor(tmp, carry)
                ab = self.gate_and(a, b)
                cin_xor = self.gate_and(carry, tmp)
                c_out = self.new_var()
                self.add_clause([-ab, c_out])
                self.add_clause([-cin_xor, c_out])
                self.add_clause([ab, cin_xor, -c_out])
            result.append(s)
            carry = c_out
        return result

    def sigma0(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 7), self.rot_right(w, 18)),
                             self.shift_right(w, 3))

    def sigma1(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 17), self.rot_right(w, 19)),
                             self.shift_right(w, 10))

    def Sigma0(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 2), self.rot_right(w, 13)),
                             self.rot_right(w, 22))

    def Sigma1(self, w):
        return self.xor_word(self.xor_word(self.rot_right(w, 6), self.rot_right(w, 11)),
                             self.rot_right(w, 25))

    def Maj_word(self, x, y, z):
        return [self.gate_maj(a, b, c) for a, b, c in zip(x, y, z)]

    def Ch_word(self, x, y, z):
        return [self.gate_ch(a, b, c) for a, b, c in zip(x, y, z)]

    def expand_W(self, M16):
        W = [None] * 64
        for i in range(16):
            W[i] = M16[i]
        for i in range(16, 64):
            s1 = self.sigma1(W[i-2])
            s0 = self.sigma0(W[i-15])
            t1 = self.add_word(s1, W[i-7])
            t2 = self.add_word(s0, W[i-16])
            W[i] = self.add_word(t1, t2)
        return W

    def compress_block(self, M16, H_state):
        W = self.expand_W(M16)
        a, b, c, d, e, f, g, h = H_state

        for i in range(64):
            S1_e = self.Sigma1(e)
            ch = self.Ch_word(e, f, g)
            ki = self.const_word(self.K[i])

            sum1 = self.add_word(h, S1_e)
            sum2 = self.add_word(ch, ki)
            sum3 = self.add_word(sum1, sum2)
            T1 = self.add_word(sum3, W[i])

            S0_a = self.Sigma0(a)
            maj = self.Maj_word(a, b, c)
            T2 = self.add_word(S0_a, maj)

            h = g
            g = f
            f = e
            e = self.add_word(d, T1)
            d = c
            c = b
            b = a
            a = self.add_word(T1, T2)

        final_work = [a, b, c, d, e, f, g, h]
        new_H = []
        for i in range(8):
            new_H.append(self.add_word(H_state[i], final_work[i]))
        return new_H

    def build_sha256d_header80(self):
        # header80 = 20 words = 80 bytes
        hdr_words = []
        for wi in range(20):
            wbits = []
            for bi in range(32):
                v = self.new_var(f"hdr_w{wi}_b{bi}")
                wbits.append(v)
            hdr_words.append(wbits)

        # -------- SHA256(header80) = 2 blocks --------
        block1 = hdr_words[0:16]

        # block2 words:
        # w0..w3 = hdr16..hdr19
        # w4 = 0x80000000
        # w5..w13 = 0
        # w14 = 0
        # w15 = 0x00000280 (len 640 bits)
        block2 = []
        block2.extend(hdr_words[16:20])                  # w0..w3
        block2.append(self.const_word(0x80000000))       # w4
        for _ in range(5, 14):                           # w5..w13 (9 words)  <-- FIX
            block2.append(self.const_word(0))
        block2.append(self.const_word(0))                # w14
        block2.append(self.const_word(0x00000280))       # w15
        assert len(block2) == 16

        H0 = [self.const_word(x) for x in self.H_init]
        H1 = self.compress_block(block1, H0)
        H2 = self.compress_block(block2, H1)             # digest1 (8 words)

        # -------- SHA256(digest1) = 1 block --------
        # block3 words:
        # w0..w7 = digest1
        # w8 = 0x80000000
        # w9..w13 = 0
        # w14 = 0
        # w15 = 0x00000100 (len 256 bits)
        block3 = []
        block3.extend(H2)                                # w0..w7
        block3.append(self.const_word(0x80000000))       # w8
        for _ in range(9, 14):                           # w9..w13 (5 words)  <-- FIX
            block3.append(self.const_word(0))
        block3.append(self.const_word(0))                # w14
        block3.append(self.const_word(0x00000100))       # w15
        assert len(block3) == 16

        H0b = [self.const_word(x) for x in self.H_init]
        H3 = self.compress_block(block3, H0b)            # final digest (8 words)

        for wi in range(8):
            for bi, var in enumerate(H3[wi]):
                self.var_map[f"hash_w{wi}_b{bi}"] = var

        for bi, var in enumerate(hdr_words[19]):
            self.var_map[f"nonce_b{bi}"] = var

        return hdr_words, H3

    def save_dimacs(self, filename):
        with open(filename, 'w') as f:
            f.write(f"p cnf {self.var_count} {len(self.clauses)}\n")
            for clause in self.clauses:
                f.write(" ".join(map(str, clause)) + " 0\n")
        with open(filename + ".map", "w") as f:
            for k, v in self.var_map.items():
                f.write(f"{k}:{v}\n")

# ============================================================
# SAT with "heurística 100 adaptada" = phase hints
# ============================================================

def parse_cnf_and_build_phases(cnf_text: str):
    nvars = 0
    clauses = []
    clause = []
    pos = None
    neg = None
    forced = {}

    for line in cnf_text.splitlines():
        line = line.strip()
        if not line or line[0] in 'c%':
            continue
        if line.startswith('p cnf'):
            parts = line.split()
            nvars = int(parts[2])
            pos = [0.0] * (nvars + 1)
            neg = [0.0] * (nvars + 1)
            continue

        for tok in line.split():
            try:
                lit = int(tok)
            except:
                continue
            if lit == 0:
                if clause:
                    clauses.append(clause)
                    w = 1.0 / (len(clause) ** 1.5)
                    for L in clause:
                        v = abs(L)
                        if L > 0:
                            pos[v] += w
                        else:
                            neg[v] += w
                    if len(clause) == 1:
                        L = clause[0]
                        forced[abs(L)] = (L > 0)
                    clause = []
            else:
                clause.append(lit)

    if clause:
        clauses.append(clause)
        w = 1.0 / (len(clause) ** 1.5)
        for L in clause:
            v = abs(L)
            if L > 0:
                pos[v] += w
            else:
                neg[v] += w
        if len(clause) == 1:
            L = clause[0]
            forced[abs(L)] = (L > 0)

    phases = []
    for v in range(1, nvars + 1):
        if v in forced:
            phases.append(v if forced[v] else -v)
        else:
            if pos[v] > 0 and neg[v] == 0:
                phases.append(v)
            elif neg[v] > 0 and pos[v] == 0:
                phases.append(-v)
            else:
                phases.append(v if pos[v] >= neg[v] else -v)

    return nvars, clauses, phases

def solve_with_pysat_phases(nvars, clauses, phases, solver_name='g3'):
    s = PySolver(name=solver_name)
    for c in clauses:
        s.add_clause(c)

    used_phases = False
    try:
        s.set_phases(phases)
        used_phases = True
    except Exception:
        pass

    t0 = time.time()
    ok = s.solve()
    t1 = time.time()

    if not ok:
        s.delete()
        return "UNSAT", None, (t1 - t0), used_phases

    model = s.get_model()
    s.delete()
    asig = {abs(lit): (lit > 0) for lit in model}
    return "SAT", asig, (t1 - t0), used_phases

# ============================================================
# DIMACS writer/reader (FIXED)
# ============================================================

def save_dimacs_result(base, asig_final, nvars, chunk=15):
    out = f"{base}_DIMACS_resultado.txt"
    lines = ["s SAT"]
    lits = []
    for v in range(1, nvars + 1):
        val = asig_final.get(v, True)
        lits.append(str(v if val else -v))
    for i in range(0, len(lits), chunk):
        chunk_lits = lits[i:i+chunk]
        lines.append("v " + " ".join(chunk_lits) + " 0")
    text = "\n".join(lines)
    with open(out, "w") as f:
        f.write(text)
    return out, text

class BitcoinSATModel:
    def __init__(self, map_text: str, dimacs_result_text: str):
        self.var_by_name = {}
        for line in map_text.splitlines():
            if ':' not in line:
                continue
            k, v = line.strip().split(':', 1)
            try:
                v = int(v)
            except:
                continue
            self.var_by_name[k] = v

        self.assign = {}
        for line in dimacs_result_text.splitlines():
            line = line.strip()
            if not line.startswith("v "):
                continue
            parts = line.split()
            lits = parts[1:]
            if lits and lits[-1] == "0":
                lits = lits[:-1]
            for lit_str in lits:
                try:
                    lit = int(lit_str)
                except:
                    continue
                self.assign[abs(lit)] = (lit > 0)

    def _word_from_bits(self, prefix: str, wi: int) -> int:
        val = 0
        for bi in range(32):
            var = self.var_by_name.get(f"{prefix}{wi}_b{bi}")
            if var and self.assign.get(var, False):
                val |= (1 << bi)
        return val

    def get_header80_bytes(self) -> bytes:
        b = bytearray()
        for wi in range(20):
            w = self._word_from_bits("hdr_w", wi)
            b.extend(w.to_bytes(4, 'big'))
        return bytes(b)

    def get_hash_bytes(self) -> bytes:
        b = bytearray()
        for wi in range(8):
            w = self._word_from_bits("hash_w", wi)
            b.extend(w.to_bytes(4, 'big'))
        return bytes(b)

# ============================================================
# Mining loop (Bitcoin PoW real)
# ============================================================

def mine_pow_by_bits(header80_base: bytes, bits_hex: str, max_seconds=10.0, report_every=200000):
    bits_hex = bits_hex.lower().strip()
    if not re.fullmatch(r"[0-9a-f]{8}", bits_hex):
        raise ValueError("bits_hex debe ser 8 hex (ej: 1d00ffff)")

    bits32 = int(bits_hex, 16)
    exp = (bits32 >> 24) & 0xff
    mant = bits32 & 0x007fffff
    if mant == 0:
        raise ValueError("bits compact inválido (mantisa=0). NO uses 1d800000. Usa 1d00ffff o 207fffff.")

    target = bits_compact_to_target_int(bits32)
    target_hex = target_int_to_hex_be(target)

    print("\n" + "="*70)
    print("⛏️  MINING (Bitcoin real): SHA256d(header80) <= target(bits)")
    print("="*70)
    print("bits:", bits_hex, "target_be:", target_hex)

    header = bytearray(header80_base)
    start_nonce = int.from_bytes(header[76:80], 'little', signed=False)
    print("Nonce inicial (LE):", start_nonce)

    t0 = time.time()
    tries = 0
    best_zeros = -1
    best_hash = None
    best_header = None

    nonce = start_nonce
    while True:
        header[76:80] = nonce.to_bytes(4, 'little', signed=False)
        h = sha256d(bytes(header))
        h_int_le = int.from_bytes(h, 'little')
        tries += 1

        zeros = count_leading_zero_bits_display(h)
        if zeros > best_zeros:
            best_zeros = zeros
            best_hash = h
            best_header = bytes(header)

        if h_int_le <= target:
            t1 = time.time()
            print("\n🎉 FOUND POW!")
            print("Tiempo:", f"{t1 - t0:.3f}s", "intentos:", tries)
            print("Header80:", bytes(header).hex())
            print("Hash raw:", h.hex())
            print("Hash display:", hash_display_hex(h))
            print("Leading zeros (display):", zeros)
            return bytes(header), h

        if tries % report_every == 0:
            elapsed = time.time() - t0
            print(f"tries={tries} elapsed={elapsed:.1f}s bestZeros={best_zeros} bestHashDisplay={hash_display_hex(best_hash)[:24]}...")
            if elapsed >= max_seconds:
                print("\n⏱️ Tiempo límite alcanzado.")
                print("Mejor hasta ahora:")
                print("bestZeros:", best_zeros)
                print("bestHeader80:", best_header.hex())
                print("bestHash display:", hash_display_hex(best_hash))
                return None, None

        nonce = (nonce + 1) & 0xffffffff

# ============================================================
# PIPELINE RUN
# ============================================================

print("="*70)
print("  ⛏️ BITCOIN 100% PIPELINE (CORREGIDO FINAL)")
print("  CNF universal SHA256d(80B) + PySAT phase-hints + mining por target bits")
print("="*70)

# Build CNF
t_build0 = time.time()
builder = CNFBuilderSHA256d80()
builder.build_sha256d_header80()
builder.save_dimacs("bitcoin_sha256d_80B.cnf")
t_build1 = time.time()
print(f"\nCNF generado: vars={builder.var_count}, clauses={len(builder.clauses)} | build={t_build1-t_build0:.2f}s")

with open("bitcoin_sha256d_80B.cnf", "r") as f:
    cnf_text = f.read()
with open("bitcoin_sha256d_80B.cnf.map", "r") as f:
    map_text = f.read()

# Parse + phases
print("\n[1/3] Parse CNF + construir phase-hints...")
t_p0 = time.time()
nvars, clauses, phases = parse_cnf_and_build_phases(cnf_text)
t_p1 = time.time()
print(f"    nvars={nvars} clauses={len(clauses)} phases={len(phases)} time={t_p1-t_p0:.2f}s")

# Solve
print("\n[2/3] PySAT solve con phase-hints...")
res, asig, t_solve, used_phases = solve_with_pysat_phases(nvars, clauses, phases, solver_name='g3')
print(f"    res={res} solve_time={t_solve:.2f}s used_phases={used_phases}")
if res != "SAT":
    raise RuntimeError("UNSAT (inesperado en CNF universal).")

# Save DIMACS
print("\n[3/3] Guardando modelo DIMACS (FIXED)...")
out_file, dimacs_text = save_dimacs_result("bitcoin_sha256d_80B", asig, nvars, chunk=15)
print("    ->", out_file)

# Verify model
model = BitcoinSATModel(map_text, dimacs_text)
header80 = model.get_header80_bytes()
hash_cnf = model.get_hash_bytes()
hash_real = sha256d(header80)

print("\n" + "="*70)
print("✅ Verificación SHA256d(header80) del modelo SAT")
print("="*70)
print("Header80:", header80.hex())
print("Hash CNF :", hash_cnf.hex())
print("Hash real:", hash_real.hex())
print("Hash display:", hash_display_hex(hash_real))
print("Coinciden:", hash_cnf == hash_real)

if hash_cnf != hash_real:
    print("\n❌ Si esto sigue false, algo más está mal (pero con este fix normalmente queda True).")

# Mining interactive
print("\n" + "="*70)
print("⛏️ MINERÍA REAL por bits compact (Bitcoin)")
print("Ejemplos:")
print("  - Genesis bits: 1d00ffff")
print("  - Fácil: 207fffff")
print("="*70)

while True:
    bits_hex = input("\nBITS compact (8 hex) o 'q': ").strip()
    if bits_hex.lower() == 'q':
        break
    max_s = input("max_seconds (ej 5, 10, 60, 500): ").strip()
    try:
        max_seconds = float(max_s)
    except:
        max_seconds = 10.0

    try:
        found_header, found_hash = mine_pow_by_bits(header80, bits_hex, max_seconds=max_seconds, report_every=200000)
    except Exception as e:
        print("Error:", e)
        continue

    if found_header is not None:
        print("\n✅ SOLUCIÓN PoW encontrada")
        print("Header80:", found_header.hex())
        print("Hash display:", hash_display_hex(found_hash))
        print("Leading zeros (display):", count_leading_zero_bits_display(found_hash))
    else:
        print("\n⚠️ No encontrado en este tiempo. Sube max_seconds o baja dificultad.")


  ⛏️ BITCOIN 100% PIPELINE (CORREGIDO FINAL)
  CNF universal SHA256d(80B) + PySAT phase-hints + mining por target bits

CNF generado: vars=347704, clauses=1203840 | build=2.63s

[1/3] Parse CNF + construir phase-hints...
    nvars=347704 clauses=1203840 phases=347704 time=3.33s

[2/3] PySAT solve con phase-hints...
    res=SAT solve_time=27.33s used_phases=True

[3/3] Guardando modelo DIMACS (FIXED)...
    -> bitcoin_sha256d_80B_DIMACS_resultado.txt

✅ Verificación SHA256d(header80) del modelo SAT
Header80: 0d5452657c5f22e0d240f335aaf33db322a1a72431770f37d31b9f42420c2a09d49c87327e4098a86b77e4b999ce5f0b7e0cf488e80dc2b0361bacddf650efdd80716c18817295a29164e8640fa4ca54
Hash CNF : f4008cb5bcdcf764c167ca59179e128b4fbd26432e26f9d612451f70d12cf5ba
Hash real: f4008cb5bcdcf764c167ca59179e128b4fbd26432e26f9d612451f70d12cf5ba
Hash display: baf52cd1701f4512d6f9262e4326bd4f8b129e1759ca67c164f7dcbcb58c00f4
Coinciden: True

⛏️ MINERÍA REAL por bits compact (Bitcoin)
Ejemplos:
  - Genesis bits: 1d00fff